In [14]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [15]:
# Define our pet table
class Pet(Base):
    __tablename__ = 'pet'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    type = Column(String)
    age = Column(Integer)

In [16]:
# Right now, this table only exists in python and not in the actual database
Base.metadata.tables

immutabledict({'pet': Table('pet', MetaData(bind=None), Column('id', Integer(), table=<pet>, primary_key=True, nullable=False), Column('name', String(), table=<pet>), Column('type', String(), table=<pet>), Column('age', Integer(), table=<pet>), schema=None)})

In [17]:
# Create our database engine
engine = create_engine('sqlite:///pets.sqlite')

In [18]:
# This is where we create our tables in the database
Base.metadata.create_all(engine)

In [19]:
# The ORM’s “handle” to the database is the Session.
from sqlalchemy.orm import Session
session = Session(engine)

## Create Data

In [20]:
# Note that adding to the session does not update the table. It queues up those queries.
session.add(Pet(name='Justin Timbersnake', type='snek', age=2))
session.add(Pet(name='Pawtrick Stewart', type='good boy', age=10))
session.add(Pet(name='Godzilla', type='iguana', age=1))
session.add(Pet(name='Marshmallow', type='polar bear', age=4))

In [21]:
# The data hasn't been added yet
engine.execute('select * from pet').fetchall()

[(1, 'Justin Timbersnake', 'snek', 2),
 (2, 'Pawtrick Stewart', 'good boy', 10),
 (3, 'Godzilla', 'iguana', 1),
 (4, 'Marshmallow', 'polar bear', 4),
 (5, 'Justin Timbersnake', 'snek', 2),
 (6, 'Pawtrick Stewart', 'good boy', 10),
 (7, 'Godzilla', 'iguana', 1),
 (8, 'Marshmallow', 'polar bear', 4)]

In [22]:
# We can use the new attribute to see the queue of data ready to go into the database
session.new

IdentitySet([<__main__.Pet object at 0x000001D9E585C5E0>, <__main__.Pet object at 0x000001D9E585C550>, <__main__.Pet object at 0x000001D9E585C760>, <__main__.Pet object at 0x000001D9E585C700>])

In [23]:
# commit() flushes whatever remaining changes remain to the database, and commits the transaction.
session.commit()

In [24]:
# Nothing new to add
session.new

IdentitySet([])

In [25]:
# query the database
session.query(Pet.name, Pet.type, Pet.age).all()

[('Justin Timbersnake', 'snek', 2),
 ('Pawtrick Stewart', 'good boy', 10),
 ('Godzilla', 'iguana', 1),
 ('Marshmallow', 'polar bear', 4),
 ('Justin Timbersnake', 'snek', 2),
 ('Pawtrick Stewart', 'good boy', 10),
 ('Godzilla', 'iguana', 1),
 ('Marshmallow', 'polar bear', 4),
 ('Justin Timbersnake', 'snek', 2),
 ('Pawtrick Stewart', 'good boy', 10),
 ('Godzilla', 'iguana', 1),
 ('Marshmallow', 'polar bear', 4)]

## Update Data

In [26]:
# Create a query and then run update on it
pet = session.query(Pet).filter_by(name="Marshmallow").first()
pet.age += 1

In [27]:
# For modifications, we can use the dirty attribute
session.dirty

IdentitySet([<__main__.Pet object at 0x000001D9E58686A0>])

In [28]:
# Commit Transaction
session.commit()

In [29]:
# Session is up-to-date
session.dirty

IdentitySet([])

In [30]:
session.query(Pet.id, Pet.name, Pet.type, Pet.age).all()

[(1, 'Justin Timbersnake', 'snek', 2),
 (2, 'Pawtrick Stewart', 'good boy', 10),
 (3, 'Godzilla', 'iguana', 1),
 (4, 'Marshmallow', 'polar bear', 5),
 (5, 'Justin Timbersnake', 'snek', 2),
 (6, 'Pawtrick Stewart', 'good boy', 10),
 (7, 'Godzilla', 'iguana', 1),
 (8, 'Marshmallow', 'polar bear', 4),
 (9, 'Justin Timbersnake', 'snek', 2),
 (10, 'Pawtrick Stewart', 'good boy', 10),
 (11, 'Godzilla', 'iguana', 1),
 (12, 'Marshmallow', 'polar bear', 4)]

## Delete Data

In [31]:
# Create a query and then delete the row collected
pet = session.query(Pet).filter_by(id=4).one()
session.delete(pet)

In [32]:
session.deleted

IdentitySet([<__main__.Pet object at 0x000001D9E58686A0>])

In [33]:
session.commit()

In [34]:
session.deleted

IdentitySet([])

In [35]:
session.query(Pet.id, Pet.name, Pet.type, Pet.age).all()

[(1, 'Justin Timbersnake', 'snek', 2),
 (2, 'Pawtrick Stewart', 'good boy', 10),
 (3, 'Godzilla', 'iguana', 1),
 (5, 'Justin Timbersnake', 'snek', 2),
 (6, 'Pawtrick Stewart', 'good boy', 10),
 (7, 'Godzilla', 'iguana', 1),
 (8, 'Marshmallow', 'polar bear', 4),
 (9, 'Justin Timbersnake', 'snek', 2),
 (10, 'Pawtrick Stewart', 'good boy', 10),
 (11, 'Godzilla', 'iguana', 1),
 (12, 'Marshmallow', 'polar bear', 4)]

In [36]:
# Close the session
session.close()